# PRÁCTICA DE SIMULACIÓN - VACUNACION 2.0
### Alejandro Enríquez 
### Ingeniería en Ciencias de la Computación
### 10mo Ciclo

Objetivo: en base a la práctica del COVID vacunación, realizar el proceso de interrupción en base a que el 5% de los pacientes que están en la sala de espera presentan complicaciones de salud y deben ser trasladados inmediatamente al centro de salud más cercano y ser atendidos. Realizar este proceso y generar una segunda linea de simulación para el traslado, atención y alta del paciente que presenta complicaciones, en base a los siguientes datos:

- Traslado de la ambulancia: 5-15 minutos
- Atención centro de salud: 5-24 horas
- Alta: 1-2 horas
- El 10% de las personas que presentan complicaciones fallecen

Generar gráficas que indiquen las personas que presentaron complicaciones en base a los tiempos, estado de las personas y respuesta

In [1]:
# Imports
import simpy
import random
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
# Instanciar Variables
horas_diarias = 9
minutos_diarios = horas_diarias * 60

TIEMPO_TOTAL = minutos_diarios * 1
NUM_MEDICOS = 6
TIEMPO_VACUNACION = 5 #Hasta 10
INTERVALO_LLEGADA = 5
POST_VACUNA = 20
CERTIFICACION = 2 #Hasta 3
TOTAL_DIAS = 100
ESPERA_INICIAL = 6

vacunados = 0
no_vacunados = 0
pacientes = 0
bandera = 1
num_muertos=0

proceso = pd.DataFrame(columns=['dia','pacientes','vacunados','no_vacunados'])

In [16]:
# Clase 
class Vacunacion(object):
    def __init__(self, environment, num_medicos, tiempo_vacunacion, paciente):
        self.env = environment
        self.medicos = simpy.Resource(environment, num_medicos)
        self.revision = env.process(self.revisar(paciente))
        self.numero_graves = 0
        self.numero_muertos = 0
        self.falla = False
        env.process(self.complicacion())
        
    def vacunar(self, paciente):
        complicaciones = random.randint(1,100)
        global vacunados
        global no_vacunados
        global bandera
        if(complicaciones<95):
            tVacunacion = random.randint(TIEMPO_VACUNACION, TIEMPO_VACUNACION+5)
            print("El paciente %s empieza la vacunación a la hora: %.2f." %(paciente, env.now))
            yield self.env.timeout(tVacunacion)
            vacunados+=1
            bandera = 1
        else:
            print("*** El %s No puede vacunarse" % (paciente))
            no_vacunados+=1
            bandera = 0
            
    def revisar(self, paciente):
        global bandera
        print('Empieza espera: %s a la hora %.2f.' % (paciente, env.now))
        tCertificacion = random.randint(CERTIFICACION, CERTIFICACION+1)
        if bandera == 1:
            if random.randint(1,100) < 5:
                self.complicacion()
            else:
                try:
                    yield env.timeout(tCertificacion + POST_VACUNA)
                    print("[%s]  se retira vacunado y certificado a las %.2f." % (paciente, env.now))
                except simpy.Interrupt:
                    print("[%s] tuvo complicaciones a las %.2f." % (paciente, env.now))
                    self.numero_graves += 1
                    self.falla = false
                
        else:
            print("[%s]  se retira sin ser vacunado a las %.2f." % (paciente, env.now))
            
    def complicacion(self):
        print('Entra a complicacion')
        global num_muertos
        while True:
            traslado_ambulancia = random.randint(5,15)
            atencion_medica = random.randint(300, 1440)
            alta = random.randint(60, 120)
            fallece = random.randint(1,10)
            print("Existen complicaciones a las %.2f." % env.now)
            yield self.env.timeout(traslado_ambulancia + atencion_medica + alta)
            if fallece == 1:
                num_muertos += 1
                print('El paciente falleció a las %.2f.' % env.now)
            self.revision.interrupt()

In [17]:
# Procesos
def llegada_paciente(env, nombre, vacunacion):
    print("Llega paciente: %s a la hora %.2f." % (nombre, env.now))
    with vacunacion.medicos.request() as medico:
        yield medico
        #print('Empieza vacunación: %s a la hora %.2f.' % (nombre, env.now))
        yield env.process(vacunacion.vacunar(nombre))
        print('La vacunacion de %s terminó a la hora: %.2f.'%(nombre, env.now))
        env.process(vacunacion.revisar(nombre))
        print("[%s]  se retira a las %.2f." % (nombre, env.now))
        
def ejecutar_simulacion(env, num_medicos, tiempo_vacunacion, intervalo):
    global pacientes
    global ESPERA_INICIAL
    paciente = 'Paciente'
    vacunacion = Vacunacion(env, num_medicos, tiempo_vacunacion, paciente)
    for i in range(ESPERA_INICIAL):
        env.process(llegada_paciente(env, 'Paciente-%d' % (i+1), vacunacion))
        
    while True:
        yield env.timeout(random.randint(intervalo-3, intervalo+2))
        i+=1   
        env.process(llegada_paciente(env, 'Paciente-%d' % (i+1), vacunacion))
        pacientes = i+1

In [18]:
# Simulación
print("Iniciando vacunacion")
random.seed(76)

for i in range(TOTAL_DIAS):
    env = simpy.Environment()
    print("###Los médicos del día ", i, " son: ", NUM_MEDICOS)
    env.process(ejecutar_simulacion(env, NUM_MEDICOS, TIEMPO_VACUNACION, INTERVALO_LLEGADA))
    env.run(until=minutos_diarios)
    dia = [i+1, pacientes, vacunados, no_vacunados]
    proceso.loc[len(proceso)] = dia
    vacunados = 0
    no_vacunados = 0
    print("FALLECIDOS: ", num_muertos)
    if i >= 30:
        NUM_MEDICOS = 10
        INTERVALO_LLEGADA = 4
    else:
        NUM_MEDICOS = 6
        INTERVALO_LLEGADA = 5
            

Iniciando vacunacion
###Los médicos del día  0  son:  6
Empieza espera: Paciente a la hora 0.00.
Entra a complicacion
Existen complicaciones a las 0.00.
Llega paciente: Paciente-1 a la hora 0.00.
Llega paciente: Paciente-2 a la hora 0.00.
Llega paciente: Paciente-3 a la hora 0.00.
Llega paciente: Paciente-4 a la hora 0.00.
Llega paciente: Paciente-5 a la hora 0.00.
Llega paciente: Paciente-6 a la hora 0.00.
El paciente Paciente-1 empieza la vacunación a la hora: 0.00.
El paciente Paciente-2 empieza la vacunación a la hora: 0.00.
El paciente Paciente-3 empieza la vacunación a la hora: 0.00.
El paciente Paciente-4 empieza la vacunación a la hora: 0.00.
El paciente Paciente-5 empieza la vacunación a la hora: 0.00.
El paciente Paciente-6 empieza la vacunación a la hora: 0.00.
Llega paciente: Paciente-7 a la hora 4.00.
Llega paciente: Paciente-8 a la hora 6.00.
La vacunacion de Paciente-3 terminó a la hora: 6.00.
[Paciente-3]  se retira a las 6.00.
Empieza espera: Paciente-3 a la hora 6.00.

*** El Paciente-16 No puede vacunarse
La vacunacion de Paciente-16 terminó a la hora: 46.00.
[Paciente-16]  se retira a las 46.00.
Empieza espera: Paciente-16 a la hora 46.00.
[Paciente-16]  se retira sin ser vacunado a las 46.00.
[Paciente-10]  se retira vacunado y certificado a las 48.00.
Llega paciente: Paciente-17 a la hora 48.00.
El paciente Paciente-17 empieza la vacunación a la hora: 48.00.
La vacunacion de Paciente-14 terminó a la hora: 50.00.
[Paciente-14]  se retira a las 50.00.
Empieza espera: Paciente-14 a la hora 50.00.
[Paciente-11]  se retira vacunado y certificado a las 51.00.
Llega paciente: Paciente-18 a la hora 55.00.
El paciente Paciente-18 empieza la vacunación a la hora: 55.00.
La vacunacion de Paciente-17 terminó a la hora: 57.00.
[Paciente-17]  se retira a las 57.00.
Empieza espera: Paciente-17 a la hora 57.00.
Llega paciente: Paciente-19 a la hora 58.00.
*** El Paciente-19 No puede vacunarse
La vacunacion de Paciente-19 terminó a la hora: 58.00.
[Paciente-19]  

[Paciente-117]  se retira vacunado y certificado a las 534.00.
Llega paciente: Paciente-125 a la hora 535.00.
El paciente Paciente-125 empieza la vacunación a la hora: 535.00.
Llega paciente: Paciente-126 a la hora 537.00.
El paciente Paciente-126 empieza la vacunación a la hora: 537.00.
[Paciente-118]  se retira vacunado y certificado a las 538.00.
Llega paciente: Paciente-127 a la hora 539.00.
El paciente Paciente-127 empieza la vacunación a la hora: 539.00.
FALLECIDOS:  0
###Los médicos del día  4  son:  6
Empieza espera: Paciente a la hora 0.00.
Entra a complicacion
Existen complicaciones a las 0.00.
Llega paciente: Paciente-1 a la hora 0.00.
Llega paciente: Paciente-2 a la hora 0.00.
Llega paciente: Paciente-3 a la hora 0.00.
Llega paciente: Paciente-4 a la hora 0.00.
Llega paciente: Paciente-5 a la hora 0.00.
Llega paciente: Paciente-6 a la hora 0.00.
*** El Paciente-1 No puede vacunarse
El paciente Paciente-2 empieza la vacunación a la hora: 0.00.
El paciente Paciente-3 empieza 

Empieza espera: Paciente-5 a la hora 6.00.
Llega paciente: Paciente-7 a la hora 7.00.
El paciente Paciente-7 empieza la vacunación a la hora: 7.00.
La vacunacion de Paciente-2 terminó a la hora: 7.00.
[Paciente-2]  se retira a las 7.00.
Empieza espera: Paciente-2 a la hora 7.00.
La vacunacion de Paciente-4 terminó a la hora: 8.00.
[Paciente-4]  se retira a las 8.00.
Empieza espera: Paciente-4 a la hora 8.00.
La vacunacion de Paciente-6 terminó a la hora: 9.00.
[Paciente-6]  se retira a las 9.00.
Empieza espera: Paciente-6 a la hora 9.00.
Llega paciente: Paciente-8 a la hora 10.00.
El paciente Paciente-8 empieza la vacunación a la hora: 10.00.
Llega paciente: Paciente-9 a la hora 15.00.
El paciente Paciente-9 empieza la vacunación a la hora: 15.00.
La vacunacion de Paciente-7 terminó a la hora: 17.00.
[Paciente-7]  se retira a las 17.00.
Empieza espera: Paciente-7 a la hora 17.00.
La vacunacion de Paciente-8 terminó a la hora: 17.00.
[Paciente-8]  se retira a las 17.00.
Empieza espera: 

RuntimeError: <Process(revisar) object at 0x1f6da796d90> has terminated and cannot be interrupted.

In [ ]:
# Gráfica y resultados
fig = plt.figure(figsize=(20,10))

ax = fig.add_axes([0,0,1,1])
ax.bar(proceso['dia'], proceso['vacunados'])
plt.title('Proceso de vacunación por días')
plt.show()

ax1 = plt.figure(figsize=(20,10))
total_poblacion = 17700000 * 80 / 100
total_junta = 14211 * 80 / 100
total = sum(proceso['vacunados'])
total_noVacuna = sum(proceso['no_vacunados'])

labels = ['Total Junta', 'Simulación Vacunados', 'Simulación No Vacunados']
datos = [total_junta, total, total_noVacuna]

plt.bar(labels, datos)